In [ ]:
from google.colab import drive
import os

# 1. Mounting Google Drive: This allows Colab to access files in your Google Drive
drive.mount('/content/drive')

In [ ]:
# 2. Tell Colab where to find your assignment files and where to save your work

# TODO: Enter the relative path in your Google Drive of the assignment.
FOLDERNAME = None # e.g. 'cs7643/ps0/'

assert FOLDERNAME is not None, "[!] Enter the foldername."
working_directory = os.path.join("/content/drive/MyDrive/", FOLDERNAME)
assert os.path.exists(working_directory), "Make sure your FOLDERNAME is correct"

In [ ]:
%cd $working_directory

# Collect Submission - Zip + Generate PDF

Run this notebook once you have completed all the other notebooks.

It will:

* Generate a zip file of your code (`.py` and `.ipynb`) called `hw1_code_submission.zip`.
* Convert all notebooks into a single PDF file called `hw1_notebook_submission.pdf`.

If your submission for this step was successful, you should see the following display message:

`### Done! Please submit hw1_code_submission.zip and hw1_notebook_submission.pdf to Gradescope. ###`

Make sure to download the zip and pdf file locally to your computer, then submit to Gradescope. Congrats on succesfully completing the assignment!

In [ ]:
# Specify the notebook files you want to convert and merge
notebook_files = ['main.ipynb']
code_files = ['configs/**', 'models/**', 'optimizer/**', 'data/get_data.sh', 'utils.py']
patterns_to_ignore=["**/__pycache__", "**/__pycache__/**"]

submission_pdf = 'hw1_notebook_submission.pdf'
submission_zip = 'hw1_code_submission.zip'

In [ ]:
# Install requirements
!sudo apt-get install texlive texlive-latex-extra pandoc texlive-xetex texlive-fonts-recommended texlive-plain-generic
!pip install PyPDF2

In [ ]:
import os
import zipfile
import subprocess
import glob
from PyPDF2 import PdfMerger

def convert_notebooks_to_pdf(notebook_files):
    pdf_files = []
    for notebook in notebook_files:
        pdf_file = notebook.replace('.ipynb', '.pdf')
        command = f"jupyter nbconvert --to pdf {notebook}"
        try:
            subprocess.run(command, shell=True, check=True)
            pdf_files.append(pdf_file)
            print(f"Successfully converted {notebook} to PDF")
        except subprocess.CalledProcessError as e:
            print(e)
            print(f"Failed to convert {notebook} to PDF")
    return pdf_files

def merge_pdfs(pdf_files, output_file):
    merger = PdfMerger()
    for pdf in pdf_files:
        merger.append(pdf)
    merger.write(output_file)
    merger.close()
    print(f"Successfully merged PDFs into {output_file}")

def cleanup(pdf_files):
    for pdf in pdf_files:
        os.remove(pdf)
        print(f"Removed temporary file: {pdf}")

def zip_folder(folder_path, zipf: zipfile.ZipFile):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            arcname = os.path.relpath(full_path, os.path.dirname(folder_path))
            zipf.write(full_path, arcname)

def create_zip_archive(patterns_to_zip: list[str], zip_filename: str, patterns_to_ignore=None):
    with zipfile.ZipFile(zip_filename, 'w') as zip_file:
        for pattern in patterns_to_zip:
            filepaths = glob.glob(pattern, recursive=True)
            if not filepaths:
                print(f"Warning: No files found matching pattern: {pattern}")
                continue
            for filepath in filepaths:
                if patterns_to_ignore:
                    if any(glob.fnmatch.fnmatch(filepath, ignore_pattern) for ignore_pattern in patterns_to_ignore):
                        print(f"Ignored {filepath}")
                        continue
                zip_file.write(filepath, os.path.relpath(filepath))
                print(f"Added {filepath} to the zip archive")

assert os.path.normpath(os.getcwd()) == os.path.normpath(working_directory), f"Make sure to cd into {working_directory}, you are currently in {os.getcwd()}"
# Convert notebooks to PDFs
pdf_files = convert_notebooks_to_pdf(notebook_files)

# Merge PDFs
merge_pdfs(pdf_files, submission_pdf)

# Clean up individual PDF files
cleanup(pdf_files)

# Create zip archive
zip_filename = submission_zip
create_zip_archive(code_files, zip_filename, patterns_to_ignore=patterns_to_ignore)

print(f"###\nDone! Please submit {zip_filename} and {submission_pdf} to GradeScope")